# Find n-syll phrases

In [4]:
import sys; sys.path.insert(0,'/Users/ryan/github/cadence/')
import sys; sys.path.insert(0,'/Users/ryan/github/lltk/')
from cadence.imports import *

from ipywidgets import interact, interact_manual, interactive
import plotnine as p9

import lltk

In [5]:
txt=s="""And spends his prodigal wits in bootless rhyme"""

In [6]:
LINE_END_MIN_SYLL=10
LINE_END_MAX_SYLL=10
LINE_END_NUM_SYLL=10
    
def to_end(linepart_df, syll_len=LINE_END_NUM_SYLL,default=pd.DataFrame()):
    
    try:
        nsyll=linepart_df.iloc[0].linepart_num_syll
    except (IndexError,KeyError,AttributeError) as e:
        return default
    
    if nsyll<syll_len:
        return default
    elif nsyll==syll_len:
        return linepart_df
    elif nsyll>syll_len:
        # only 
        grps = reversed(list(linepart_df.groupby('word_i')))
        nsyllsofar=0
        gsofar=[]
        for wi,dfwrd in grps:
            gsofar+=[dfwrd]
            nsyllsofar+=len(dfwrd[dfwrd.word_ipa_i==1])
            if nsyllsofar==syll_len and len(gsofar):
                return pd.concat(reversed(gsofar))
    
    return default


def scan_iter_ends(txt_or_df, syll_len=LINE_END_NUM_SYLL, num_proc=1,progress=True,lim=False,**kwargs):
    iterr_i=scan_iter(
        txt_or_df,
        progress=progress,
        num_proc=num_proc,
        min_len=None,
        max_len=None,
        lim=None,
        **kwargs
    )
    iterr_o=pmap_iter(
        to_end,
        iterr_i,
        kwargs=dict(syll_len=syll_len),
        num_proc=1,
        progress=False
    )

    i=0
    for x in iterr_o:
        if len(x):
            i+=1
            yield x
            if lim and i>=lim: break
            

def scan_ends(txt_or_df,**kwargs):
    o=list(scan_iter_ends(txt_or_df,**kwargs))
    return pd.concat(o) if len(o) else pd.DataFrame()

In [7]:
# odf=scan_ends(txt,num_proc=1,lim=10).reset_index()
# odf['id']=list(zip(odf.stanza_i,odf.line_i,odf.linepart_i))
# odf.id.value_counts()

## Parsing

In [8]:
def parse_iter_ends(txt_or_df, syll_len=LINE_END_NUM_SYLL, num_proc=1,progress=True,lim=None,**kwargs):
    iterr_i=scan_iter_ends(txt_or_df,progress=progress,syll_len=syll_len,lim=None,num_proc=num_proc,**kwargs)
    iterr_o=pmap_iter(get_parse, iterr_i, num_proc=1, progress=False, kwargs=kwargs)
    i=0
    for x in iterr_o:
        if len(x):
            i+=1
            yield x
            if lim and i>=lim: break
            

In [9]:
def parse_ends(txt_or_df, syll_len=LINE_END_NUM_SYLL, num_proc=1,progress=True,**kwargs):
    o=list(parse_iter_ends(txt_or_df, syll_len=syll_len, num_proc=num_proc, progress=progress, **kwargs))
    return setindex(pd.concat(o)) if len(o) else pd.DataFrame()

In [10]:
# df=parse_ends(txt,lim=200)
# df

In [13]:
# odf=parse_ends(txt,syll_len=10,lim=100,num_proc=1).reset_index()
# odf['id']=list(zip(odf.stanza_i,odf.line_i,odf.linepart_i))
# odf.id.value_counts()

In [15]:
print(sonnet)


How heavy do I journey on the way,
When what I seek, my weary travel's end,
Doth teach that ease and that repose to say
'Thus far the miles are measured from thy friend!'
The beast that bears me, tired with my woe,
Plods dully on, to bear that weight in me,
As if by some instinct the wretch did know
His rider loved not speed, being made from thee:
The bloody spur cannot provoke him on
That sometimes anger thrusts into his hide;
Which heavily he answers with a groan,
More sharp to me than spurring to his side;
For that same groan doth put this in my mind;
My grief lies onward and my joy behind.



In [16]:
to_lines(parse_ends(sonnet))

Scanning lines [x1]: 100%|████████████████████████████████████████████████| 15/15 [00:00<00:00, 110.93it/s]


prom_strength  ...  num_parses
stanza_i line_i linepart_i linepart_str                                   parse_rank is_troch parse_i parse      parse_str                                      combo_stress        combo_ipa                                          combo_i parse_is_bounded parse_bounded_by                 ...            
1        1      1          How heavy do I journey on the way,             1          0        1       wswswswsws how|HEA|vy|DO|i|JOUR|ney|ON*|the|WAY           U PU P U PU U U P   haʊ 'hɛ.viː 'duː aɪ 'ʤɛː.niː ɑn ðə 'weɪ            1       False                                        2.0  ...         1.0
         2      5          the wretch did know His rider loved not speed, 1          0        1       wswswswsws the|WRETCH|did|KNOW|his|RI|der|LOVED|not|SPEED U P U P U PU P U P  ðə 'ɹɛtʃ dɪd 'noʊ hɪz 'raɪ.dɛː 'lʌvd nɑt 'spiːd    1       False                                        1.0  ...         1.0
                7          That sometimes anger thrusts into his hide;    1          0        2       wswswswsws that|SO|metimes*|AN|ger|THRUSTS|in|TO|his|HIDE U PS PU P UP U P    ðət 'səm.`taɪmz 'æŋ.gɛː 'θrʌsts ɪn.'tuː hɪz 'haɪd  2       False                                        3.0  ...         2.0
                                                                          2          1        1       swsswswsws THAT|so|METIMES.AN*|ger|THRUSTS|in|TO|his|HIDE P UP PU P UP U P    'ðæt səm.'taɪmz 'æŋ.gɛː 'θrʌsts ɪn.'tuː hɪz 'haɪd  1       False                                        3.0  ...         2.0
                8          Which heavily he answers with a groan,         1          0        1       wswswswsws which|HEA|vi|LY*|he|AN|swers|WITH*|a|GROAN     U PUU U PU U U P    wɪʧ 'hɛ.və.liː hiː 'æn.sɛːz wɪð eɪ 'groʊn          1       False                                        2.0  ...         3.0
                                                                          2          0        2       wswwsswsws which|HEA|vi.ly*|HE.AN*|swers|WITH*|a|GROAN    U PUU P PU U U P    wɪʧ 'hɛ.və.liː 'hiː 'æn.sɛːz wɪð eɪ 'groʊn         2       False                                        2.0  ...         3.0
                                                                          3          0        3       wswwswswws which|HEA|vi.ly*|HE|an*|SWERS*|with.a|GROAN    U PUU P PU U U P    wɪʧ 'hɛ.və.liː 'hiː 'æn.sɛːz wɪð eɪ 'groʊn         2       False                                        2.0  ...         3.0
                9          More sharp to me than spurring to his side;    1          0        1       wswswswsws more|SHARP|to|ME*|than|SPUR|ring|TO*|his|SIDE  U P U U U PU U U P  mɔːr 'ʃɑrp tuː miː ðæn 'spɛː.ɪŋ tuː hɪz 'saɪd      1       False                                        1.0  ...         1.0
                10         For that same groan doth put this in my mind;  1          0        1       wswswswsws for|THAT|same*|GROAN|doth|PUT|this|IN|my|MIND  U P P P U P U P U P fɔːr 'ðæt 'seɪm 'groʊn dɔːθ 'pʊt ðɪs 'ɪn maɪ 'm... 1       False                                        0.0  ...         2.0
                                                                          2          0        2       wwsswswsws for.that|SAME.GROAN*|doth|PUT|this|IN|my|MIND  U U P P U P U P U P fɔːr ðət 'seɪm 'groʊn dɔːθ 'pʊt ðɪs 'ɪn maɪ 'maɪnd 2       False                                        0.0  ...         2.0
                11         My grief lies onward and my joy behind.        1          0        1       wswswswsws my|GRIEF|lies*|ON|ward|AND*|my|JOY|be|HIND     U P P PU U U P UP   maɪ 'griːf 'laɪz 'ɔːn.wɛːd ænd maɪ 'ʤɔɪ bɪ.'haɪnd  1       False                                        2.0  ...         1.0

[11 rows x 23 columns]

In [129]:
def parse_ends_corpus(C,lim=None,line_lim=100,force=False,num_proc=1,shuffle=True,progress_inner=False):
    ofnfn=os.path.join('data',C.name,f'data.parse_ends.dc')
    # num_proc=1
    with dc.Cache(ofnfn) as od:
        objs=[
            (t.id,t.path_txt)
            for t in C.texts() if os.path.exists(t.path_txt)
            if force or not t.id in od
        ][:lim]
        if shuffle: random.shuffle(objs)
        iterr_o=pmap_iter(
            do_parse_ends_corpus,
            objs,
            num_proc=num_proc,
            kwargs=dict(progress=progress_inner,line_lim=line_lim,num_proc=1),
            progress=False,
        )
        iterr_o2=tqdm(
            iterr_o,
            total=len(objs),
            desc=f'Parsing line ends in {C.name} [x{num_proc}]'
        )
        for idx,x in iterr_o2:
            iterr_o2.set_description(idx)
            od[idx]=x
    
def do_parse_ends_corpus(obj,line_lim=100,progress=False,**kwargs):
    idx,path=obj
    try:
        odf=parse_ends(path, lim=line_lim, progress=progress, **kwargs)
    except Exception as e:
        odf=pd.DataFrame()
    return (idx,odf)

In [130]:
# txt="""
# and spends his mighty wits in bootless rhyme.
# and spends his prodigal wits in bootless rhyme.
# and spends his whimsical wits in bootless rhyme.
# """


In [131]:
# to_lines(parse(txt))

In [132]:
# !rm -rf data/Saintsbury/data.parse_ends.dc

In [ ]:
# # %%timeit -n1 -r1
C=lltk.load('Chadwyck')
parse_ends_corpus(C,lim=None,num_proc=8,force=False,line_lim=500,shuffle=True,progress_inner=True)

Parsing line ends in Chadwyck [x8]:   0%|                                            | 0/8 [00:00<?, ?it/s]